In [ ]:
"""
Dirichlet Variational Auto-Encoder (Dir-VAE) implementation for MNIST
Based on "Autoencodeing Variational Inference for Topic Model" (ICLR2017)
"""

from __future__ import print_function

import argparse
import os
import sys
from dataclasses import dataclass
from typing import Tuple

import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image


@dataclass
class Config:
    """Configuration class for Dir-VAE training"""

    batch_size: int = 256
    epochs: int = 10
    learning_rate: float = 1e-3
    no_cuda: bool = False
    seed: int = 10
    log_interval: int = 2
    category: int = 10  # Number of latent categories (K)
    alpha: float = 0.3  # Dirichlet hyperparameter
    data_dir: str = "./data"
    output_dir: str = "./image"

    # Network architecture parameters
    encoder_channels: int = 64
    decoder_channels: int = 64
    input_channels: int = 1
    latent_dim: int = 1024
    hidden_dim: int = 512


def create_argument_parser() -> argparse.ArgumentParser:
    """Create and configure argument parser"""
    parser = argparse.ArgumentParser(description="Dir-VAE MNIST Example")
    parser.add_argument(
        "--batch-size",
        type=int,
        default=256,
        metavar="N",
        help="input batch size for training (default: 256)",
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=10,
        metavar="N",
        help="number of epochs to train (default: 10)",
    )
    parser.add_argument(
        "--learning-rate",
        type=float,
        default=1e-3,
        help="learning rate (default: 1e-3)",
    )
    parser.add_argument(
        "--no-cuda", action="store_true", default=False, help="disable CUDA training"
    )
    parser.add_argument(
        "--seed", type=int, default=10, metavar="S", help="random seed (default: 10)"
    )
    parser.add_argument(
        "--log-interval",
        type=int,
        default=2,
        metavar="N",
        help="how many batches to wait before logging training status",
    )
    parser.add_argument(
        "--category",
        type=int,
        default=10,
        metavar="K",
        help="the number of categories in the dataset",
    )
    parser.add_argument(
        "--alpha",
        type=float,
        default=0.3,
        help="Dirichlet hyperparameter alpha (default: 0.3)",
    )
    parser.add_argument(
        "--data-dir",
        type=str,
        default="./data",
        help="directory for dataset (default: ./data)",
    )
    parser.add_argument(
        "--output-dir",
        type=str,
        default="./image",
        help="directory for output images (default: ./image)",
    )
    return parser


def setup_device_and_seed(config: Config) -> torch.device:
    """Setup device and random seeds"""
    # Set random seeds for reproducibility
    torch.manual_seed(config.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(config.seed)

    # Determine device
    use_cuda = not config.no_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    print(f"Using device: {device}")
    return device


def create_data_loaders(
    config: Config,
) -> Tuple[torch.utils.data.DataLoader, torch.utils.data.DataLoader]:
    """Create train and test data loaders"""
    # Create data directory if it doesn't exist
    os.makedirs(config.data_dir, exist_ok=True)

    # Data loader kwargs
    kwargs = (
        {"num_workers": 1, "pin_memory": True}
        if not config.no_cuda and torch.cuda.is_available()
        else {}
    )

    # Create datasets and loaders
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(
            config.data_dir, train=True, download=True, transform=transforms.ToTensor()
        ),
        batch_size=config.batch_size,
        shuffle=True,
        **kwargs,
    )

    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(config.data_dir, train=False, transform=transforms.ToTensor()),
        batch_size=config.batch_size,
        shuffle=False,
        **kwargs,
    )

    return train_loader, test_loader


def compute_dirichlet_prior(
    K: int, alpha: float, device: torch.device
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Compute Dirichlet prior parameters using Laplace approximation.

    Args:
        K: Number of categories
        alpha: Dirichlet hyperparameter
        device: torch device

    Returns:
        Tuple of (mean, variance) tensors for the approximated normal distribution
    """
    # Laplace approximation to convert Dirichlet to multivariate normal
    a = torch.full((1, K), alpha, dtype=torch.float, device=device)
    mean = a.log().t() - a.log().mean(1, keepdim=True)
    var = ((1 - 2.0 / K) * a.reciprocal()).t() + (1.0 / K**2) * a.reciprocal().sum(
        1, keepdim=True
    )
    return mean.t(), var.t()


class DirVAEEncoder(nn.Module):
    """Encoder part of Dir-VAE"""

    def __init__(self, config: Config):
        super(DirVAEEncoder, self).__init__()
        ndf = config.encoder_channels
        nc = config.input_channels

        self.conv_layers = nn.Sequential(
            # input: (nc) x 28 x 28
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state: (ndf) x 14 x 14
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state: (ndf*2) x 7 x 7
            nn.Conv2d(ndf * 2, ndf * 4, 3, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state: (ndf*4) x 4 x 4
            nn.Conv2d(ndf * 4, config.latent_dim, 4, 1, 0, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.fc1 = nn.Linear(config.latent_dim, config.hidden_dim)
        self.fc_mu = nn.Linear(config.hidden_dim, config.category)
        self.fc_logvar = nn.Linear(config.hidden_dim, config.category)

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        conv_out = self.conv_layers(x)
        h1 = self.fc1(conv_out.view(conv_out.size(0), -1))
        mu = self.fc_mu(h1)
        logvar = self.fc_logvar(h1)
        return mu, logvar


class DirVAEDecoder(nn.Module):
    """Decoder part of Dir-VAE"""

    def __init__(self, config: Config):
        super(DirVAEDecoder, self).__init__()
        ngf = config.decoder_channels
        nc = config.input_channels

        self.fc_decode = nn.Linear(config.category, config.hidden_dim)
        self.fc_deconv = nn.Linear(config.hidden_dim, config.latent_dim)

        self.deconv_layers = nn.Sequential(
            # input: latent_dim x 1 x 1
            nn.ConvTranspose2d(config.latent_dim, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state: (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 3, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state: (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state: (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, nc, 4, 2, 1, bias=False),
            nn.Sigmoid(),
            # output: (nc) x 32 x 32
        )

        self.relu = nn.ReLU()

    def forward(self, gauss_z: torch.Tensor) -> torch.Tensor:
        # Apply softmax to satisfy simplex constraint (Dirichlet distribution)
        dir_z = F.softmax(gauss_z, dim=1)

        h3 = self.relu(self.fc_decode(dir_z))
        deconv_input = self.fc_deconv(h3)
        deconv_input = deconv_input.view(-1, deconv_input.size(1), 1, 1)

        return self.deconv_layers(deconv_input)


class DirVAE(nn.Module):
    """Dirichlet Variational Auto-Encoder"""

    def __init__(self, config: Config, device: torch.device):
        super(DirVAE, self).__init__()
        self.config = config
        self.device = device

        # Initialize encoder and decoder
        self.encoder = DirVAEEncoder(config)
        self.decoder = DirVAEDecoder(config)

        # Setup Dirichlet prior
        self._setup_dirichlet_prior()

    def _setup_dirichlet_prior(self):
        """Setup Dirichlet prior parameters"""
        prior_mean, prior_var = compute_dirichlet_prior(
            self.config.category, self.config.alpha, self.device
        )
        self.register_buffer("prior_mean", prior_mean)
        self.register_buffer("prior_var", prior_var)
        self.register_buffer("prior_logvar", prior_var.log())

    def encode(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """Encode input to latent parameters"""
        return self.encoder(x)

    def decode(self, gauss_z: torch.Tensor) -> torch.Tensor:
        """Decode latent variables to reconstruction"""
        return self.decoder(gauss_z)

    def reparameterize(self, mu: torch.Tensor, logvar: torch.Tensor) -> torch.Tensor:
        """Reparameterization trick for backpropagation through stochastic nodes"""
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(
        self, x: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """Forward pass through the network"""
        mu, logvar = self.encode(x)
        gauss_z = self.reparameterize(mu, logvar)

        # gauss_z follows multivariate normal distribution
        # Applying softmax gives us Dirichlet-distributed variables
        dir_z = F.softmax(gauss_z, dim=1)
        recon_x = self.decode(gauss_z)

        return recon_x, mu, logvar, gauss_z, dir_z

    def loss_function(
        self,
        recon_x: torch.Tensor,
        x: torch.Tensor,
        mu: torch.Tensor,
        logvar: torch.Tensor,
    ) -> torch.Tensor:
        """
        Compute the loss function: reconstruction loss + KL divergence

        Args:
            recon_x: Reconstructed images
            x: Original images
            mu: Mean of latent distribution
            logvar: Log variance of latent distribution

        Returns:
            Total loss (sum over batch)
        """
        # Reconstruction loss (Binary Cross Entropy)
        BCE = F.binary_cross_entropy(
            recon_x.view(-1, 784), x.view(-1, 784), reduction="sum"
        )

        # KL divergence between Dirichlet prior and variational posterior
        # Based on the original paper: "Autoencodeing variational inference for topic model"
        prior_mean = self.prior_mean.expand_as(mu)
        prior_var = self.prior_var.expand_as(logvar)
        prior_logvar = self.prior_logvar.expand_as(logvar)

        var_division = logvar.exp() / prior_var  # Σ_0 / Σ_1
        diff = mu - prior_mean  # μ_1 - μ_0
        diff_term = diff * diff / prior_var  # (μ_1 - μ_0)² / Σ_1
        logvar_division = prior_logvar - logvar  # log|Σ_1| - log|Σ_0|

        # KL divergence
        KLD = 0.5 * (
            var_division + diff_term + logvar_division - self.config.category
        ).sum(dim=1)

        return BCE + KLD.sum()


class DirVAETrainer:
    """Trainer class for Dir-VAE"""

    def __init__(
        self,
        model: DirVAE,
        config: Config,
        device: torch.device,
        train_loader: torch.utils.data.DataLoader,
        test_loader: torch.utils.data.DataLoader,
    ):
        self.model = model
        self.config = config
        self.device = device
        self.train_loader = train_loader
        self.test_loader = test_loader

        # Initialize optimizer
        self.optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

        # Create output directory
        os.makedirs(config.output_dir, exist_ok=True)

    def train_epoch(self, epoch: int) -> float:
        """Train for one epoch"""
        self.model.train()
        train_loss = 0.0

        for batch_idx, (data, _) in enumerate(self.train_loader):
            data = data.to(self.device)

            self.optimizer.zero_grad()
            recon_batch, mu, logvar, gauss_z, dir_z = self.model(data)
            loss = self.model.loss_function(recon_batch, data, mu, logvar)

            loss.backward()
            train_loss += loss.item()
            self.optimizer.step()

            if batch_idx % self.config.log_interval == 0:
                print(
                    f"Train Epoch: {epoch} [{batch_idx * len(data):5d}/"
                    f"{len(self.train_loader.dataset)} "
                    f"({100. * batch_idx / len(self.train_loader):3.0f}%)]"
                    f"\tLoss: {loss.item() / len(data):.6f}"
                )

        avg_loss = train_loss / len(self.train_loader.dataset)
        print(f"====> Epoch: {epoch} Average loss: {avg_loss:.4f}")
        return avg_loss

    def test_epoch(self, epoch: int) -> float:
        """Test for one epoch"""
        self.model.eval()
        test_loss = 0.0

        with torch.no_grad():
            for i, (data, _) in enumerate(self.test_loader):
                data = data.to(self.device)
                recon_batch, mu, logvar, gauss_z, dir_z = self.model(data)
                loss = self.model.loss_function(recon_batch, data, mu, logvar)
                test_loss += loss.item()

                if i == 0:
                    n = min(data.size(0), 18)
                    # Properly reshape for comparison
                    comparison = torch.cat(
                        [data[:n], recon_batch.view(data.size(0), 1, 28, 28)[:n]]
                    )
                    save_image(
                        comparison.cpu(),
                        os.path.join(self.config.output_dir, f"recon_{epoch}.png"),
                        nrow=n,
                    )

        avg_loss = test_loss / len(self.test_loader.dataset)
        print(f"====> Test set loss: {avg_loss:.4f}")
        return avg_loss

    def generate_samples(self, epoch: int, num_samples: int = 64):
        """Generate samples from the model"""
        self.model.eval()
        with torch.no_grad():
            # Sample from latent space
            sample = torch.randn(num_samples, self.config.category).to(self.device)
            sample = self.model.decode(sample).cpu()
            save_image(
                sample.view(num_samples, 1, 28, 28),
                os.path.join(self.config.output_dir, f"sample_{epoch}.png"),
            )

    def train(self):
        """Main training loop"""
        print("Starting training...")
        print(f"Configuration: {self.config}")

        for epoch in range(1, self.config.epochs + 1):
            train_loss = self.train_epoch(epoch)
            test_loss = self.test_epoch(epoch)
            self.generate_samples(epoch)


"""Main function"""
# Parse arguments
parser = create_argument_parser()
args, unknown = parser.parse_known_args()

# Create configuration
config = Config(
    batch_size=args.batch_size,
    epochs=args.epochs,
    learning_rate=args.learning_rate,
    no_cuda=args.no_cuda,
    seed=args.seed,
    log_interval=args.log_interval,
    category=args.category,
    alpha=args.alpha,
    data_dir=args.data_dir,
    output_dir=args.output_dir,
)

try:
    # Setup device and random seeds
    device = setup_device_and_seed(config)

    # Create data loaders
    train_loader, test_loader = create_data_loaders(config)

    # Create model
    model = DirVAE(config, device).to(device)
    print(
        f"Model created with {sum(p.numel() for p in model.parameters() if p.requires_grad)} trainable parameters"
    )

    # Create trainer and start training
    trainer = DirVAETrainer(model, config, device, train_loader, test_loader)
    trainer.train()

except Exception as e:
    print(f"Error during training: {e}", file=sys.stderr)
    sys.exit(1)


Using device: cpu


100%|██████████| 9.91M/9.91M [00:02<00:00, 4.94MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 130kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.22MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 1.72MB/s]


Model created with 15784468 trainable parameters
Starting training...
Configuration: Config(batch_size=256, epochs=10, learning_rate=0.001, no_cuda=False, seed=10, log_interval=2, category=10, alpha=0.3, data_dir='./data', output_dir='./image', encoder_channels=64, decoder_channels=64, input_channels=1, latent_dim=1024, hidden_dim=512)


KeyboardInterrupt: 

In [ ]:
##Gauss try (doesnt work yet)

"""
Gauss Variational Auto-Encoder (Gauss-VAE) implementation for MNIST
"""

from __future__ import print_function

import argparse
import os
import sys
from dataclasses import dataclass
from typing import Tuple

import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image


@dataclass
class Config:
    """Configuration class for Gauss-VAE training"""

    batch_size: int = 256
    epochs: int = 10
    learning_rate: float = 1e-3
    no_cuda: bool = False
    seed: int = 10
    log_interval: int = 2
    category: int = 10  # Number of latent categories (K)
    data_dir: str = "./data"
    output_dir: str = "./image"

    # Network architecture parameters
    encoder_channels: int = 64
    decoder_channels: int = 64
    input_channels: int = 1
    latent_dim: int = 1024
    hidden_dim: int = 512


def create_argument_parser() -> argparse.ArgumentParser:
    """Create and configure argument parser"""
    parser = argparse.ArgumentParser(description="Gauss-VAE MNIST Example")
    parser.add_argument(
        "--batch-size",
        type=int,
        default=256,
        metavar="N",
        help="input batch size for training (default: 256)",
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=10,
        metavar="N",
        help="number of epochs to train (default: 10)",
    )
    parser.add_argument(
        "--learning-rate",
        type=float,
        default=1e-3,
        help="learning rate (default: 1e-3)",
    )
    parser.add_argument(
        "--no-cuda", action="store_true", default=False, help="disable CUDA training"
    )
    parser.add_argument(
        "--seed", type=int, default=10, metavar="S", help="random seed (default: 10)"
    )
    parser.add_argument(
        "--log-interval",
        type=int,
        default=2,
        metavar="N",
        help="how many batches to wait before logging training status",
    )
    parser.add_argument(
        "--category",
        type=int,
        default=10,
        metavar="K",
        help="the number of categories in the dataset",
    )

    parser.add_argument(
        "--data-dir",
        type=str,
        default="./data",
        help="directory for dataset (default: ./data)",
    )
    parser.add_argument(
        "--output-dir",
        type=str,
        default="./image",
        help="directory for output images (default: ./image)",
    )
    return parser


def setup_device_and_seed(config: Config) -> torch.device:
    """Setup device and random seeds"""
    # Set random seeds for reproducibility
    torch.manual_seed(config.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(config.seed)

    # Determine device
    use_cuda = not config.no_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    print(f"Using device: {device}")
    return device


def create_data_loaders(
    config: Config,
) -> Tuple[torch.utils.data.DataLoader, torch.utils.data.DataLoader]:
    """Create train and test data loaders"""
    # Create data directory if it doesn't exist
    os.makedirs(config.data_dir, exist_ok=True)

    # Data loader kwargs
    kwargs = (
        {"num_workers": 1, "pin_memory": True}
        if not config.no_cuda and torch.cuda.is_available()
        else {}
    )

    # Create datasets and loaders
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(
            config.data_dir, train=True, download=True, transform=transforms.ToTensor()
        ),
        batch_size=config.batch_size,
        shuffle=True,
        **kwargs,
    )

    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(config.data_dir, train=False, transform=transforms.ToTensor()),
        batch_size=config.batch_size,
        shuffle=False,
        **kwargs,
    )

    return train_loader, test_loader



class GaussVAEEncoder(nn.Module):
    """Encoder part of Gauss-VAE"""

    def __init__(self, config: Config):
        super(GaussVAEEncoder, self).__init__()
        ndf = config.encoder_channels
        nc = config.input_channels

        self.conv_layers = nn.Sequential(
            # input: (nc) x 28 x 28
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state: (ndf) x 14 x 14
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state: (ndf*2) x 7 x 7
            nn.Conv2d(ndf * 2, ndf * 4, 3, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state: (ndf*4) x 4 x 4
            nn.Conv2d(ndf * 4, config.latent_dim, 4, 1, 0, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.fc1 = nn.Linear(config.latent_dim, config.hidden_dim)
        self.fc_mu = nn.Linear(config.hidden_dim, config.category)
        self.fc_logvar = nn.Linear(config.hidden_dim, config.category)

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        conv_out = self.conv_layers(x)
        h1 = self.fc1(conv_out.view(conv_out.size(0), -1))
        mu = self.fc_mu(h1)
        logvar = self.fc_logvar(h1)
        return mu, logvar


class GaussVAEDecoder(nn.Module):
    """Decoder part of Gauss-VAE"""

    def __init__(self, config: Config):
        super(GaussVAEDecoder, self).__init__()
        ngf = config.decoder_channels
        nc = config.input_channels

        self.fc_decode = nn.Linear(config.category, config.hidden_dim)
        self.fc_deconv = nn.Linear(config.hidden_dim, config.latent_dim)

        self.deconv_layers = nn.Sequential(
            # input: latent_dim x 1 x 1
            nn.ConvTranspose2d(config.latent_dim, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state: (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 3, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state: (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state: (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, nc, 4, 2, 1, bias=False),
            nn.Sigmoid(),
            # output: (nc) x 32 x 32
        )

        self.relu = nn.ReLU()

    def forward(self, gauss_z: torch.Tensor) -> torch.Tensor:
        h3 = self.relu(self.fc_decode(gauss_z))
        deconv_input = self.fc_deconv(h3)
        deconv_input = deconv_input.view(-1, deconv_input.size(1), 1, 1)

        return self.deconv_layers(deconv_input)


class GaussVAE(nn.Module):
    """Gauss Variational Auto-Encoder"""

    def __init__(self, config: Config, device: torch.device):
        super(GaussVAE, self).__init__()
        self.config = config
        self.device = device

        # Initialize encoder and decoder
        self.encoder = GaussVAEEncoder(config)
        self.decoder = GaussVAEDecoder(config)

    def encode(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """Encode input to latent parameters"""
        return self.encoder(x)

    def decode(self, gauss_z: torch.Tensor) -> torch.Tensor:
        """Decode latent variables to reconstruction"""
        return self.decoder(gauss_z)

    def reparameterize(self, mu: torch.Tensor, logvar: torch.Tensor) -> torch.Tensor:
        """Reparameterization trick for backpropagation through stochastic nodes"""
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(
        self, x: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """Forward pass through the network"""
        mu, logvar = self.encode(x)
        gauss_z = self.reparameterize(mu, logvar)
        recon_x = self.decode(gauss_z)

        return recon_x, mu, logvar, gauss_z

    def loss_function(
        self,
        recon_x: torch.Tensor,
        x: torch.Tensor,
        mu: torch.Tensor,
        logvar: torch.Tensor,
    ) -> torch.Tensor:
        """
        Compute the loss function: reconstruction loss + KL divergence

        Args:
            recon_x: Reconstructed images
            x: Original images
            mu: Mean of latent distribution
            logvar: Log variance of latent distribution

        Returns:
            Total loss (sum over batch)
        """
        # Reconstruction loss (Binary Cross Entropy)
        BCE = F.binary_cross_entropy(
            recon_x.view(-1, 784), x.view(-1, 784), reduction="sum"
        )


        # KL divergence
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return BCE + KLD


class GaussVAETrainer:
    """Trainer class for Gauss-VAE"""

    def __init__(
        self,
        model: GaussVAE,
        config: Config,
        device: torch.device,
        train_loader: torch.utils.data.DataLoader,
        test_loader: torch.utils.data.DataLoader,
    ):
        self.model = model
        self.config = config
        self.device = device
        self.train_loader = train_loader
        self.test_loader = test_loader

        # Initialize optimizer
        self.optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

        # Create output directory
        os.makedirs(config.output_dir, exist_ok=True)

    def train_epoch(self, epoch: int) -> float:
        """Train for one epoch"""
        self.model.train()
        train_loss = 0.0

        for batch_idx, (data, _) in enumerate(self.train_loader):
            data = data.to(self.device)

            self.optimizer.zero_grad()
            recon_batch, mu, logvar, gauss_z= self.model(data)
            loss = self.model.loss_function(recon_batch, data, mu, logvar)

            loss.backward()
            train_loss += loss.item()
            self.optimizer.step()

            if batch_idx % self.config.log_interval == 0:
                print(
                    f"Train Epoch: {epoch} [{batch_idx * len(data):5d}/"
                    f"{len(self.train_loader.dataset)} "
                    f"({100. * batch_idx / len(self.train_loader):3.0f}%)]"
                    f"\tLoss: {loss.item() / len(data):.6f}"
                )

        avg_loss = train_loss / len(self.train_loader.dataset)
        print(f"====> Epoch: {epoch} Average loss: {avg_loss:.4f}")
        return avg_loss

    def test_epoch(self, epoch: int) -> float:
        """Test for one epoch"""
        self.model.eval()
        test_loss = 0.0

        with torch.no_grad():
            for i, (data, _) in enumerate(self.test_loader):
                data = data.to(self.device)
                recon_batch, mu, logvar, gauss_z = self.model(data)
                loss = self.model.loss_function(recon_batch, data, mu, logvar)
                test_loss += loss.item()

                if i == 0:
                    n = min(data.size(0), 18)
                    # Properly reshape for comparison
                    comparison = torch.cat(
                        [data[:n], recon_batch.view(data.size(0), 1, 28, 28)[:n]]
                    )
                    save_image(
                        comparison.cpu(),
                        os.path.join(self.config.output_dir, f"recon_{epoch}.png"),
                        nrow=n,
                    )

        avg_loss = test_loss / len(self.test_loader.dataset)
        print(f"====> Test set loss: {avg_loss:.4f}")
        return avg_loss

    def generate_samples(self, epoch: int, num_samples: int = 64):
      self.model.eval()
      with torch.no_grad():
        # sample from standard Gaussian prior
        z = torch.randn(num_samples, self.config.latent_dim, device=self.device)
        sample = self.model.decode(z).cpu()
        save_image(
            sample.view(num_samples, 1, 28, 28),
            os.path.join(self.config.output_dir, f"sample_{epoch}.png"),
            nrow=8
        )

    def train(self):
        """Main training loop"""
        print("Starting training...")
        print(f"Configuration: {self.config}")

        for epoch in range(1, self.config.epochs + 1):
            train_loss = self.train_epoch(epoch)
            test_loss = self.test_epoch(epoch)
            self.generate_samples(epoch)


def main(argv=None):
    parser = create_argument_parser()

    # When running in notebook, argv should be empty to use defaults.
    if argv is None:
        # Detect running in IPython/Jupyter and avoid taking notebook argv
        if "ipykernel" in sys.modules:
            args = parser.parse_args([])
        else:
            args = parser.parse_args()
    else:
        args = parser.parse_args(argv)

    config = Config(
        batch_size=args.batch_size,
        epochs=args.epochs,
        learning_rate=args.learning_rate,
        no_cuda=args.no_cuda,
        seed=args.seed,
        log_interval=args.log_interval,
        category=args.category,
        data_dir=args.data_dir,
        output_dir=args.output_dir,
    )

    device = setup_device_and_seed(config)
    train_loader, test_loader = create_data_loaders(config)

    model = GaussVAE(config, device).to(device)
    n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Model created with {n_params} trainable parameters")

    trainer = GaussVAETrainer(model, config, device, train_loader, test_loader)
    trainer.train()


if __name__ == "__main__":
    main()

Using device: cuda
Model created with 15784468 trainable parameters
Starting training...
Configuration: Config(batch_size=256, epochs=10, learning_rate=0.001, no_cuda=False, seed=10, log_interval=2, category=10, data_dir='./data', output_dir='./image', encoder_channels=64, decoder_channels=64, input_channels=1, latent_dim=1024, hidden_dim=512)
Train Epoch: 1 [    0/60000 (  0%)]	Loss: 664.023376
Train Epoch: 1 [  512/60000 (  1%)]	Loss: 1024.685303
Train Epoch: 1 [ 1024/60000 (  2%)]	Loss: 387.771484
Train Epoch: 1 [ 1536/60000 (  3%)]	Loss: 277.235809
Train Epoch: 1 [ 2048/60000 (  3%)]	Loss: 259.702667
Train Epoch: 1 [ 2560/60000 (  4%)]	Loss: 240.780960
Train Epoch: 1 [ 3072/60000 (  5%)]	Loss: 236.565521
Train Epoch: 1 [ 3584/60000 (  6%)]	Loss: 226.450684
Train Epoch: 1 [ 4096/60000 (  7%)]	Loss: 227.766541
Train Epoch: 1 [ 4608/60000 (  8%)]	Loss: 213.367126
Train Epoch: 1 [ 5120/60000 (  9%)]	Loss: 211.869003
Train Epoch: 1 [ 5632/60000 (  9%)]	Loss: 208.035919
Train Epoch: 1 [ 

In [6]:
"""
Forsøg på DIR fra den nye artikel
"""

from __future__ import print_function

import argparse
import os
import sys
from dataclasses import dataclass
from typing import Tuple

import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

EPS = 1e-8

@dataclass
class Config:
    """Configuration class for Dir-VAE training"""

    batch_size: int = 256
    epochs: int = 10
    learning_rate: float = 1e-3
    no_cuda: bool = False
    seed: int = 10
    log_interval: int = 2
    category: int = 10  # Number of latent categories (K)
    alpha: float = 0.3  # Dirichlet hyperparameter
    data_dir: str = "./data"
    output_dir: str = "./image"
    beta=1.0

    # Network architecture parameters
    encoder_channels: int = 64
    decoder_channels: int = 64
    input_channels: int = 1
    latent_dim: int = 1024
    hidden_dim: int = 512


def create_argument_parser() -> argparse.ArgumentParser:
    """Create and configure argument parser"""
    parser = argparse.ArgumentParser(description="Dir-VAE MNIST Example")
    parser.add_argument(
        "--batch-size",
        type=int,
        default=256,
        metavar="N",
        help="input batch size for training (default: 256)",
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=10,
        metavar="N",
        help="number of epochs to train (default: 10)",
    )
    parser.add_argument(
        "--learning-rate",
        type=float,
        default=1e-3,
        help="learning rate (default: 1e-3)",
    )
    parser.add_argument(
        "--no-cuda", action="store_true", default=False, help="disable CUDA training"
    )
    parser.add_argument(
        "--seed", type=int, default=10, metavar="S", help="random seed (default: 10)"
    )
    parser.add_argument(
        "--log-interval",
        type=int,
        default=2,
        metavar="N",
        help="how many batches to wait before logging training status",
    )
    parser.add_argument(
        "--category",
        type=int,
        default=10,
        metavar="K",
        help="the number of categories in the dataset",
    )
    parser.add_argument(
        "--alpha",
        type=float,
        default=0.3,
        help="Dirichlet hyperparameter alpha (default: 0.3)",
    )
    parser.add_argument(
        "--data-dir",
        type=str,
        default="./data",
        help="directory for dataset (default: ./data)",
    )
    parser.add_argument(
        "--output-dir",
        type=str,
        default="./image",
        help="directory for output images (default: ./image)",
    )
    return parser


def setup_device_and_seed(config: Config) -> torch.device:
    """Setup device and random seeds"""
    # Set random seeds for reproducibility
    torch.manual_seed(config.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(config.seed)

    # Determine device
    use_cuda = not config.no_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    print(f"Using device: {device}")
    return device


def create_data_loaders(
    config: Config,
) -> Tuple[torch.utils.data.DataLoader, torch.utils.data.DataLoader]:
    """Create train and test data loaders"""
    # Create data directory if it doesn't exist
    os.makedirs(config.data_dir, exist_ok=True)

    # Data loader kwargs
    kwargs = (
        {"num_workers": 1, "pin_memory": True}
        if not config.no_cuda and torch.cuda.is_available()
        else {}
    )

    # Create datasets and loaders
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(
            config.data_dir, train=True, download=True, transform=transforms.ToTensor()
        ),
        batch_size=config.batch_size,
        shuffle=True,
        **kwargs,
    )

    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(config.data_dir, train=False, transform=transforms.ToTensor()),
        batch_size=config.batch_size,
        shuffle=False,
        **kwargs,
    )

    return train_loader, test_loader


def compute_dirichlet_prior(
    K: int, alpha: float, device: torch.device
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Compute Dirichlet prior parameters using Laplace approximation.

    Args:
        K: Number of categories
        alpha: Dirichlet hyperparameter
        device: torch device

    Returns:
        Tuple of (mean, variance) tensors for the approximated normal distribution
    """
    # Laplace approximation to convert Dirichlet to multivariate normal
    a = torch.full((1, K), alpha, dtype=torch.float, device=device)
    mean = a.log().t() - a.log().mean(1, keepdim=True)
    var = ((1 - 2.0 / K) * a.reciprocal()).t() + (1.0 / K**2) * a.reciprocal().sum(
        1, keepdim=True
    )
    return mean.t(), var.t()


class DirVAEEncoder(nn.Module):
    """Encoder part of Dir-VAE"""

    def __init__(self, config: Config):
        super(DirVAEEncoder, self).__init__()
        ndf = config.encoder_channels
        nc = config.input_channels

        self.conv_layers = nn.Sequential(
            # input: (nc) x 28 x 28
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state: (ndf) x 14 x 14
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state: (ndf*2) x 7 x 7
            nn.Conv2d(ndf * 2, ndf * 4, 3, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state: (ndf*4) x 4 x 4
            nn.Conv2d(ndf * 4, config.latent_dim, 4, 1, 0, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.fc1 = nn.Linear(config.latent_dim, config.hidden_dim)
        self.fc_mu = nn.Linear(config.hidden_dim, config.category)
        self.fc_logvar = nn.Linear(config.hidden_dim, config.category)
        self.softplus = nn.Softplus()
        self.fc_alpha = nn.Linear(config.hidden_dim, config.category)

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        x = x.view(x.size(0), -1)
        conv_out = self.conv_layers(x)

        h1 = self.fc1(conv_out.view(conv_out.size(0), -1))
        alpha_hat=self.softplus(self.fc_alpha(h1))
        return alpha_hat


class DirVAEDecoder(nn.Module):
    """Decoder part of Dir-VAE"""

    def __init__(self, config: Config):
        super(DirVAEDecoder, self).__init__()
        ngf = config.decoder_channels
        nc = config.input_channels

        self.fc_decode = nn.Linear(config.category, config.hidden_dim)
        self.fc_deconv = nn.Linear(config.hidden_dim, config.latent_dim)

        self.deconv_layers = nn.Sequential(
            # input: latent_dim x 1 x 1
            nn.ConvTranspose2d(config.latent_dim, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state: (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 3, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state: (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state: (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, nc, 4, 2, 1, bias=False),
            nn.Sigmoid(),
            # output: (nc) x 32 x 32
        )

        self.relu = nn.ReLU()

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        # Apply softmax to satisfy simplex constraint (Dirichlet distribution)

        h3 = self.fc_decode(z)
        deconv_input = self.fc_deconv(h3)
        deconv_input = deconv_input.view(-1, deconv_input.size(1), 1, 1)

        return self.deconv_layers(deconv_input)


class DirVAE(nn.Module):
    """Dirichlet Variational Auto-Encoder"""

    def __init__(self, config: Config, device: torch.device):
        super(DirVAE, self).__init__()
        self.config = config
        self.device = device

        # Initialize encoder and decoder
        self.encoder = DirVAEEncoder(config)
        self.decoder = DirVAEDecoder(config)
        # Initialize prior_alpha
        if not hasattr(config, "prior_alpha") or config.alpha is None:
            # default weak symmetric prior
            prior_alpha = torch.ones(config.category) * 0.98
        else:
            # use config.alpha (scalar or tensor)
            prior_alpha = torch.tensor(config.alpha).repeat(config.category)

        self.register_buffer('prior_alpha', prior_alpha.float())
        self.register_buffer('beta', torch.tensor(float(config.beta)))


    def encode(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """Encode input to latent parameters"""
        return self.encoder(x)

    def decode(self, z: torch.Tensor) -> torch.Tensor:
        """Decode latent variables to reconstruction"""
        return self.decoder(z)

    def inverse_gamma_cdf_approx(self, u, alpha):
        """
        Approximate inverse CDF for X ~ Gamma(alpha, beta) using:
        F^{-1}(u; alpha, beta) ≈ beta^{-1} * (u * alpha * Gamma(alpha))^{1/alpha}
        u: uniform samples in (0,1), shape (batch, K)
        alpha: shape (batch, K) or (K,)
        returns: approx gamma samples shape (batch, K)
        """
        # alpha * Gamma(alpha) = alpha * exp(lgamma(alpha))
        # note: torch.lgamma for log Gamma
        # shapes broadcast
        log_gamma = torch.lgamma(alpha)
        a_gamma = alpha * torch.exp(log_gamma)  # shape (batch, K)
        # clamp u to (eps, 1)
        u = u.clamp(min=EPS, max=1.0 - 1e-12)
        base = u * a_gamma
        # to avoid negative/zero values due to numerical issues, clamp base
        base = base.clamp(min=EPS)
        # power 1/alpha
        samples = (base) ** (1.0 / alpha)
        # divide by beta (rate)
        samples = samples / (self.beta + 0.0)
        return samples

    def sample_dirichlet_from_alpha(self, alpha_hat):
        """
        Given alpha_hat (batch, K), produce reparam samples z on simplex:
          1) draw u ~ Uniform(0,1) per component
          2) approximate gamma sample via inverse Gamma CDF approx
          3) normalize v -> z = v / sum_k v_k
        Returns z (batch, K), v (batch, K), u (for reproducibility)
        """
        assert torch.all(alpha_hat > 0), "alpha_hat must be positive"
        batch = alpha_hat.shape[0]
        # Uniform draws per component
        u = torch.rand_like(alpha_hat)  # Uniform(0,1)
        v = self.inverse_gamma_cdf_approx(u, alpha_hat)
        # Normalize to get Dirichlet sample
        denom = v.sum(dim=1, keepdim=True).clamp(min=EPS)
        z = v / denom
        return z, v, u

    def forward(
        self, x: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """Forward pass through the network"""
        alpha_hat = self.encode(x)
        z, v, u = self.sample_dirichlet_from_alpha(alpha_hat)
        recon_x = self.decode(z)

        return recon_x, z,alpha_hat,v

    def loss_function(
        self,
        recon_x: torch.Tensor,
        x: torch.Tensor,
        alpha_hat: torch.Tensor,
    ) -> torch.Tensor:
        """
        Compute the loss function: reconstruction loss + KL divergence

        Args:
            recon_x: Reconstructed images
            x: Original images
            mu: Mean of latent distribution
            logvar: Log variance of latent distribution

        Returns:
            Total loss (sum over batch)
        """
        # Reconstruction loss (Binary Cross Entropy)
        recon_x = recon_x.clamp(0.0, 1.0)
        BCE = F.binary_cross_entropy(
            recon_x.view(-1, self.cfg.input_dim),
            x.view(-1, self.cfg.input_dim),
            reduction="sum"
        )


        # KL divergence
        # broadcast prior_alpha to batch if necessary
        if self.prior_alpha.dim() == 1:
            prior = self.prior_alpha.unsqueeze(0).expand_as(alpha_hat)
        else:
            prior = self.prior_alpha
        term1 = torch.lgamma(prior) - torch.lgamma(alpha_hat)
        term2 = (alpha_hat - prior) * torch.digamma(alpha_hat)
        kl_comp = term1 + term2
        kl = kl_comp.sum(dim=1)  # per example sum over K
        KLD=kl.mean()
        return BCE + KLD


class DirVAETrainer:
    """Trainer class for Dir-VAE"""

    def __init__(
        self,
        model: DirVAE,
        config: Config,
        device: torch.device,
        train_loader: torch.utils.data.DataLoader,
        test_loader: torch.utils.data.DataLoader,
    ):
        self.model = model
        self.config = config
        self.device = device
        self.train_loader = train_loader
        self.test_loader = test_loader

        # Initialize optimizer
        self.optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

        # Create output directory
        os.makedirs(config.output_dir, exist_ok=True)

    def train_epoch(self, epoch: int) -> float:
        """Train for one epoch"""
        self.model.train()
        train_loss = 0.0

        for batch_idx, (data, _) in enumerate(self.train_loader):
            data = data.to(self.device)

            self.optimizer.zero_grad()
            recon_batch, z,alpha_hat,v = self.model(data)
            loss = self.model.loss_function(recon_batch,data, alpha_hat)

            loss.backward()
            train_loss += loss.item()
            self.optimizer.step()

            if batch_idx % self.config.log_interval == 0:
                print(
                    f"Train Epoch: {epoch} [{batch_idx * len(data):5d}/"
                    f"{len(self.train_loader.dataset)} "
                    f"({100. * batch_idx / len(self.train_loader):3.0f}%)]"
                    f"\tLoss: {loss.item() / len(data):.6f}"
                )

        avg_loss = train_loss / len(self.train_loader.dataset)
        print(f"====> Epoch: {epoch} Average loss: {avg_loss:.4f}")
        return avg_loss

    def test_epoch(self, epoch: int) -> float:
        """Test for one epoch"""
        self.model.eval()
        test_loss = 0.0

        with torch.no_grad():
            for i, (data, _) in enumerate(self.test_loader):
                data = data.to(self.device)
                recon_batch,z, alpha_hat,v = self.model(data)
                loss = self.model.loss_function(recon_batch,data, alpha_hat)
                test_loss += loss.item()

                if i == 0:
                    n = min(data.size(0), 18)
                    # Properly reshape for comparison
                    comparison = torch.cat(
                        [data[:n], recon_batch.view(data.size(0), 1, 28, 28)[:n]]
                    )
                    save_image(
                        comparison.cpu(),
                        os.path.join(self.config.output_dir, f"recon_{epoch}.png"),
                        nrow=n,
                    )

        avg_loss = test_loss / len(self.test_loader.dataset)
        print(f"====> Test set loss: {avg_loss:.4f}")
        return avg_loss

    def generate_samples(self, epoch: int, num_samples: int = 64):
        """Generate samples from the model"""
        self.model.eval()
        with torch.no_grad():
            # Sample from latent space
            sample = torch.randn(num_samples, self.config.category).to(self.device)
            sample = self.model.decode(sample).cpu()
            save_image(
                sample.view(num_samples, 1, 28, 28),
                os.path.join(self.config.output_dir, f"sample_{epoch}.png"),
            )

    def train(self):
        """Main training loop"""
        print("Starting training...")
        print(f"Configuration: {self.config}")

        for epoch in range(1, self.config.epochs + 1):
            train_loss = self.train_epoch(epoch)
            test_loss = self.test_epoch(epoch)
            self.generate_samples(epoch)


"""Main function"""
# Parse arguments
parser = create_argument_parser()
args, unknown = parser.parse_known_args()

# Create configuration
config = Config(
    batch_size=args.batch_size,
    epochs=args.epochs,
    learning_rate=args.learning_rate,
    no_cuda=args.no_cuda,
    seed=args.seed,
    log_interval=args.log_interval,
    category=args.category,
    alpha=args.alpha,
    data_dir=args.data_dir,
    output_dir=args.output_dir,
)

try:
    # Setup device and random seeds
    device = setup_device_and_seed(config)

    # Create data loaders
    train_loader, test_loader = create_data_loaders(config)

    # Create model
    model = DirVAE(config, device).to(device)
    print(
        f"Model created with {sum(p.numel() for p in model.parameters() if p.requires_grad)} trainable parameters"
    )

    # Create trainer and start training
    trainer = DirVAETrainer(model, config, device, train_loader, test_loader)
    trainer.train()

except Exception as e:
    print(f"Error during training: {e}", file=sys.stderr)
    sys.exit(1)


Error during training: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/tmp/ipython-input-1253410673.py", line 511, in <cell line: 0>
    device = setup_device_and_seed(config)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1253410673.py", line 113, in setup_device_and_seed
    torch.manual_seed(config.seed)
  File "/usr/local/lib/python3.12/dist-packages/torch/_compile.py", line 53, in inner
    return disable_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py", line 929, in _fn
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/random.py", line 46, in manual_seed
    torch.cuda.manual_seed_all(seed)
  File "/usr/local/lib/python3.12/dist-packages/torch/cuda/random.py", line 131, in manual_seed_all
    _lazy_call(cb, seed_all=True)
  File "/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py", line 341, in _lazy_call
    callable()
 

TypeError: object of type 'NoneType' has no len()